# improved clearned DATASET 

# TWO categorical features coverd to ordered numbers

# add two new variable'TotalSF',and "OverallQual-S2"(= train["OverallQual"]^2)

# DEL 4 variable otalBsmtSF'] ,1stFlrSF' and '2ndFlrSF', OverallQual


# contain only response price, no boxcoxprice, logprice 



In [71]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
color = sns.color_palette() 
sns.set_style('darkgrid') 
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)



# Check for duplicates  (only test set has Id)

In [72]:
idsUnique = len(set(test.Id))
idsTotal = test.shape[0]
idsDupli = idsTotal - idsUnique
print("There are " + str(idsDupli) + " duplicate IDs for " + str(idsTotal) + " total entries")

There are 0 duplicate IDs for 1608 total entries


In [73]:
test.drop("Id", axis = 1, inplace = True)

In [74]:
print("The train data size is : {} ".format(train.shape))    
print("The test data size after dropping Id feature is : {} ".format(test.shape))

The train data size is : (804, 79) 
The test data size after dropping Id feature is : (1608, 78) 


# Imputing missing values

In [75]:
# PoolQC : data description says NA means "no pool"
all_data.loc[:, "PoolQC"] = all_data.loc[:, "PoolQC"].fillna("No")

# MiscFeature : data description says NA means "no misc feature"
all_data.loc[:, "MiscFeature"] = all_data.loc[:, "MiscFeature"].fillna("No")

# Alley : data description says NA means "no alley access"                        
all_data.loc[:, "Alley"] = all_data.loc[:, "Alley"].fillna("None") 

# Fence: data description says NA means "no fence"
all_data.loc[:, "Fence"] = all_data.loc[:, "Fence"].fillna("No")
# FireplaceQu : data description says NA means "no fireplace"
all_data.loc[:, "FireplaceQu"] = all_data.loc[:, "FireplaceQu"].fillna("No")

# LotFrontage : NA most likely means no lot frontage
all_data.loc[:, "LotFrontage"] = all_data.loc[:, "LotFrontage"].fillna(0)
# BsmtQual etc : data description says NA for basement features is "no basement"     
all_data.loc[:, "BsmtQual"] = all_data.loc[:, "BsmtQual"].fillna("No")        
all_data.loc[:, "BsmtCond"] = all_data.loc[:, "BsmtCond"].fillna("No")        
all_data.loc[:, "BsmtExposure"] = all_data.loc[:, "BsmtExposure"].fillna("No")
all_data.loc[:, "BsmtFinType1"] = all_data.loc[:, "BsmtFinType1"].fillna("No")
all_data.loc[:, "BsmtFinType2"] = all_data.loc[:, "BsmtFinType2"].fillna("No")
all_data.loc[:, "BsmtFullBath"] = all_data.loc[:, "BsmtFullBath"].fillna(0)
all_data.loc[:, "BsmtHalfBath"] = all_data.loc[:, "BsmtHalfBath"].fillna(0)


# GarageType etc : data description says NA for garage features is "no garage"
all_data.loc[:, "GarageType"] = all_data.loc[:, "GarageType"].fillna("No")
all_data.loc[:, "GarageFinish"] = all_data.loc[:, "GarageFinish"].fillna("No")
all_data.loc[:, "GarageQual"] = all_data.loc[:, "GarageQual"].fillna("No")
all_data.loc[:, "GarageCond"] = all_data.loc[:, "GarageCond"].fillna("No")
all_data.loc[:, "GarageYrBlt"] = all_data.loc[:, "GarageYrBlt"].fillna(0)  

# MasVnrType : NA most likely means no veneer
all_data.loc[:, "MasVnrType"] = all_data.loc[:, "MasVnrType"].fillna("None")
all_data.loc[:, "MasVnrArea"] = all_data.loc[:, "MasVnrArea"].fillna(0)

# Electrical: data description says NA means "no Electrical system"  
all_data.loc[:, "Electrical"] =all_data.loc[:, "Electrical"].fillna("No") 

# Transforming some numerical variables that are really categorical

In [76]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

## Overall Cond (Ordinal): Rates the overall condition of the house

all_data['OverallCond'] = all_data['OverallCond'].astype(str)


all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

all_data = all_data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                             
                             "OverallCond" : {1 :"Very Poor",2 :"Poor",3 :"Fair",4 :"Below Average",5 :"Average",
                                              6 :"Above Average",7 :"Good",8 :"Very Good",9 :"Excellent",
                                              10 :"Very Excellent"},
                             "YrSold": {2006 : "Year2006", 2007 : "Year2007", 2008 : "Year2008", 2009 : "Year2009",
                                         2010 :"Year2010"},
                             
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

In [77]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF'] 
del all_data['TotalBsmtSF']
del all_data['1stFlrSF']
del all_data['2ndFlrSF']  

In [78]:
all_data["OverallQual-s2"] = all_data["OverallQual"] ** 2
del all_data["OverallQual"]

# TWO categorical features are ordered numbers

In [79]:
cols = ('BsmtExposure','LandSlope')

for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2412, 78)


In [80]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2412, 335)


In [81]:
train_EDA = all_data[:ntrain]
test_EDA = all_data[ntrain:]
test_EDA.drop(['SalePrice'], axis=1, inplace=True)
train_EDA.head() 
train_EDA.shape

(804, 335)

In [82]:
train_EDA.to_csv("Train_new.csv")
test_EDA.to_csv("Test_new.csv")